In [1]:
import os
import pandas as pd
import numpy as np
root= 'C:/Users/DELL/Desktop/recent/研一下/学习/量化/final_project/quant_code/'
os.chdir(root)
import utils.tools as tools
import utils.calculators as calculators
import factor_gen.y as y

In [2]:
def transfer_data(sample):
    sample2 = sample.set_index(['PubDate'])
    sample2.index = pd.to_datetime(sample2.index)
    sample2 = sample2[~sample2.index.duplicated()]#去除重复索引
    sample2 = sample2.resample('D').asfreq().ffill()#对齐到每天，若对齐5分钟-5T
    sample2 = sample2.reset_index()
    return sample2

In [3]:
def get_raw_factor101():
    '''预测数据'''
    forecast_data = pd.read_csv(os.path.join(root,'raw_data','factor101','PerformanceForecast_DataYes.csv'))
    forecast_data = forecast_data.drop(columns = ['ID','RptDate','UPDATE_TIME','EndDate'])

    #处理forecast object
    dic = {'A':1, 'Q1':2, 'Q3':3, 'S1':4}
    forecast_data['ForecastObject'] = forecast_data['ForecastObject'].apply(lambda x: dic[x])

    forecast_data = forecast_data[forecast_data['PubDate']>=737061]#取时间窗  737061：2018.1.1
    forecast_data['PubDate'] = tools.Str2times(tools.ConTimeDay2(forecast_data['PubDate']))
    
    #时间对齐到日频
    forecast_data = forecast_data.groupby('Ticker').apply(lambda x:transfer_data(x))
    
    #重置索引
    forecast_data = forecast_data.reset_index(drop =True)
    print('finish preparing forecast data')
    
    '''资产负债表'''
    balancesheet_data = pd.read_csv(os.path.join(root,'raw_data','factor101','FinBalanceSheet_DataYes.csv'))

    equity = balancesheet_data[['Ticker','EndDate','PublDate','T_SH_EQUITY']]#归母净利润/总股本
    print('finish preparing balance sheet data')
    
    '''损益表'''
    incomestatement_data = pd.read_csv(os.path.join(root,'raw_data','factor101','FinIncomeStat_DataYes.csv'))
    incomestatement_data = incomestatement_data[['Ticker','EndDate','PublDate','REVENUE','T_PROFIT','DILUTED_EPS','INCOME_TAX']]
    #计算净利润
    incomestatement_data['N_PROFIT'] = incomestatement_data['T_PROFIT'] - incomestatement_data['INCOME_TAX']
    incomestatement_data.drop(columns = 'INCOME_TAX')
    #计算增长率部分
    incomestatement_data = incomestatement_data.sort_values(by =['Ticker','EndDate'],ascending =False)
    #营业收入增长率
    incomestatement_data['REVENUE_GROWTH'] = incomestatement_data['REVENUE']/incomestatement_data['REVENUE'].shift(1)
    #利润增长率
    profit_pre = incomestatement_data['T_PROFIT'].drop(0,axis = 0)
    profit_pre = np.append(np.array(profit_pre),1)
    incomestatement_data['T_PROFIT_GROWTH'] = incomestatement_data['T_PROFIT']/incomestatement_data['T_PROFIT'].shift(1)
    print('finish preparing incomestatement data')
    
    '''并表'''
    '''合并两张财务报表'''
    combine_data = pd.merge(incomestatement_data,equity,how = 'outer',on = ['Ticker','EndDate'])
    #计算ROE
    combine_data['ROE'] = combine_data['T_PROFIT']/combine_data['T_SH_EQUITY']
    combine_data = combine_data.drop(columns = 'T_SH_EQUITY')
    #保留早的PubDate
    combine_data['PubDate'] = combine_data[['PublDate_x', 'PublDate_y']].min(axis=1)
    combine_data = combine_data.drop(columns = ['PublDate_x', 'PublDate_y','EndDate'])
    #处理时间
    combine_data = combine_data[combine_data['PubDate']>=737061]#取时间窗  737061：2018.1.1
    combine_data['PubDate'] = tools.Str2times(tools.ConTimeDay2(combine_data['PubDate']))
    #时间对齐到日频
    combine_data = combine_data.groupby('Ticker').apply(lambda x:transfer_data(x))
    combine_data = combine_data.reset_index(drop =True)
    
    '''并入预期数据'''
    combine_data2 = pd.merge(combine_data,forecast_data,how = 'outer',on = ['Ticker','PubDate']) #combine_data-pubdate_x，forecast_data-pubdate_y
    #按报告期正序排序
    #combine_data2 = combine_data2.sort_values(by =['EndDate'],ascending =True)
    print('finish preparing 3 datas')
    
    return combine_data2

In [14]:
data = get_raw_factor101()

finish preparing forecast data
finish preparing balance sheet data
finish preparing incomestatement data
finish preparing 3 datas


In [15]:
data

,PubDate,Ticker,REVENUE,T_PROFIT,DILUTED_EPS,INCOME_TAX,N_PROFIT,REVENUE_GROWTH,T_PROFIT_GROWTH,ROE,...,EProfitRateFloor,EProfitRateCeiling,EProfitFloor,EProfitCeiling,EProfitParentRateFloor,EProfitParentRateCeiling,EProfitParentFloor,EProfitParentCeiling,EEPSFloor,EEPSCeiling
0,2018-03-14,1.0,1.057860e+11,3.015700e+10,1.3,6.968000e+09,2.318900e+10,1.000000,1.000000,0.135809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-03-15,1.0,1.057860e+11,3.015700e+10,1.3,6.968000e+09,2.318900e+10,3.774566,3.520135,0.135809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-03-16,1.0,1.057860e+11,3.015700e+10,1.3,6.968000e+09,2.318900e+10,3.774566,3.520135,0.135809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-03-17,1.0,1.057860e+11,3.015700e+10,1.3,6.968000e+09,2.318900e+10,3.774566,3.520135,0.135809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-03-18,1.0,1.057860e+11,3.015700e+10,1.3,6.968000e+09,2.318900e+10,3.774566,3.520135,0.135809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5902068,2022-04-27,4861.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.15,13.62,33000000.0,36000000.0,4.15,13.62,3.300000e+07,3.800000e+07,NaN,NaN
5902069,2022-04-28,4862.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.96,46.03,1.850000e+07,2.350000e+07,NaN,NaN
5902070,2022-04-28,4863.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,45.79,61.14,8.931520e+07,9.871690e+07,NaN,NaN
5902071,2022-05-05,4866.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.86,35.20,5.953390e+11,7.676000e+11,NaN,NaN


In [19]:
data[list(data.columns)[2:]] = data.groupby('Ticker').apply(lambda x:calculators.ts_Decay2(x[list(x.columns)[2:]],5)).reset_index(drop =True)

In [23]:
data['Ticker'] = data['Ticker'].astype(int)

In [21]:
y = y.get_day_rate()

500
1000
1500
2000
2500
3000
3500
4000
4500


In [22]:
y

,time,rate,code
0,2018-01-01,0.000000,1
1,2018-01-02,0.030075,1
2,2018-01-03,-0.027007,1
3,2018-01-04,-0.006002,1
4,2018-01-05,0.003774,1
...,...,...,...
1552,2022-04-02,NaN,301268
1553,2022-04-03,NaN,301268
1554,2022-04-04,NaN,301268
1555,2022-04-05,NaN,301268


In [24]:
total_data = pd.merge(data,y,how = 'inner',left_on = ['Ticker','PubDate'],right_on = ['code','time'])

In [25]:
total_data

,PubDate,Ticker,REVENUE,T_PROFIT,DILUTED_EPS,INCOME_TAX,N_PROFIT,REVENUE_GROWTH,T_PROFIT_GROWTH,ROE,...,EProfitCeiling,EProfitParentRateFloor,EProfitParentRateCeiling,EProfitParentFloor,EProfitParentCeiling,EEPSFloor,EEPSCeiling,time,rate,code
0,2018-03-14,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-14,-0.008319,1
1,2018-03-15,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-15,-0.017617,1
2,2018-03-16,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-16,-0.005978,1
3,2018-03-17,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-17,0.000000,1
4,2018-03-18,1,1.057860e+11,3.015700e+10,1.3000,6.968000e+09,2.318900e+10,3.589595,3.352126,0.135809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-03-18,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226316,2022-01-25,3043,8.658383e+08,6.428955e+07,0.4814,6.516068e+06,5.777348e+07,1.000000,1.000000,0.154077,...,NaN,82.41,105.21,80000000.0,90000000.0,NaN,NaN,2022-01-25,-0.035680,3043
2226317,2022-01-26,3043,8.658383e+08,6.428955e+07,0.4814,6.516068e+06,5.777348e+07,1.000000,1.000000,0.154077,...,NaN,82.41,105.21,80000000.0,90000000.0,NaN,NaN,2022-01-26,-0.003544,3043
2226318,2022-01-27,3043,8.658383e+08,6.428955e+07,0.4814,6.516068e+06,5.777348e+07,1.000000,1.000000,0.154077,...,NaN,82.41,105.21,80000000.0,90000000.0,NaN,NaN,2022-01-27,-0.047375,3043
2226319,2022-01-28,3043,8.658383e+08,6.428955e+07,0.4814,6.516068e+06,5.777348e+07,1.000000,1.000000,0.154077,...,NaN,82.41,105.21,80000000.0,90000000.0,NaN,NaN,2022-01-28,0.024194,3043


In [31]:
total_data = total_data.sort_values(by =['PubDate'],ascending =True)

In [32]:
total_data

,PubDate,Ticker,REVENUE,T_PROFIT,DILUTED_EPS,INCOME_TAX,N_PROFIT,REVENUE_GROWTH,T_PROFIT_GROWTH,ROE,...,EProfitCeiling,EProfitParentRateFloor,EProfitParentRateCeiling,EProfitParentFloor,EProfitParentCeiling,EEPSFloor,EEPSCeiling,time,rate,code
2152438,2018-01-02,890,2.409787e+08,5.027342e+07,0.3000,7.015349e+06,4.325807e+07,1.000000,1.000000,0.065267,...,NaN,-50.00,-30.00,9280300.0,12992500.0,NaN,NaN,2018-01-02,0.022590,890
2134744,2018-01-03,528,2.165372e+08,7.373034e+07,0.4000,1.832732e+07,5.540302e+07,0.473576,0.566520,0.036553,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-03,-0.002291,528
2152439,2018-01-03,890,2.409787e+08,5.027342e+07,0.3000,7.015349e+06,4.325807e+07,1.000000,1.000000,0.065267,...,NaN,-50.00,-30.00,9280300.0,12992500.0,NaN,NaN,2018-01-03,0.000000,890
2158981,2018-01-03,2015,4.410587e+07,-4.721419e+06,-0.0600,1.520433e+04,-4.736623e+06,0.418988,0.287987,-0.008361,...,NaN,-316.08,-290.66,-17000000.0,-15000000.0,-0.140,-0.130,2018-01-03,0.000000,2015
2189756,2018-01-03,2426,6.512137e+08,1.350038e+08,0.7100,2.146756e+07,1.135363e+08,1.000004,0.986870,0.080040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-03,0.030508,2426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889264,2022-04-06,2120,1.943388e+09,1.560825e+08,0.3400,3.030150e+07,1.257810e+08,1.000000,1.000000,0.094250,...,NaN,570.77,608.66,177000000.0,187000000.0,NaN,NaN,2022-04-06,-0.057045,2120
564656,2022-04-06,889,3.167811e+10,3.988338e+09,0.8025,5.807514e+08,3.407587e+09,1.000000,1.000000,0.158607,...,NaN,116.60,141.80,736000000.0,822400000.0,NaN,NaN,2022-04-06,0.026379,889
1960136,2022-04-06,2911,9.588322e+09,4.400670e+08,NaN,7.477157e+07,3.652954e+08,1.058682,0.838068,0.056907,...,NaN,823.00,933.00,250000000.0,280000000.0,NaN,NaN,2022-04-06,0.014478,2911
680437,2022-04-06,998,1.266212e+09,9.319958e+07,0.2600,-1.382820e+07,1.070278e+08,1.000000,1.000000,0.088960,...,NaN,67.00,101.00,100000000.0,120000000.0,NaN,NaN,2022-04-06,0.002607,998


In [34]:
X = total_data[['REVENUE',
 'T_PROFIT',
 'DILUTED_EPS',
 'INCOME_TAX',
 'N_PROFIT',
 'REVENUE_GROWTH',
 'T_PROFIT_GROWTH',
 'ROE',
 'ForcastType',
 'FiscalPeriod',
 'ForecastObject',
 'EEarningRateFloor',
 'EEarningRateCeiling',
 'EEarningFloor',
 'EEarningCeiling',
 'EProfitRateFloor',
 'EProfitRateCeiling',
 'EProfitFloor',
 'EProfitCeiling',
 'EProfitParentRateFloor',
 'EProfitParentRateCeiling',
 'EProfitParentFloor',
 'EProfitParentCeiling',
 'EEPSFloor',
 'EEPSCeiling']]
Y = total_data[['rate']]

In [38]:
split = int(len(total_data)*0.7)
X_train = X.iloc[:,:split]
X_test = X.iloc[:,split:]
Y_train = Y.iloc[:,:split]
Y_test = Y.iloc[:,split:]

In [ ]:
from gplearn.genetic import SymbolicTransformer
st = SymbolicTransformer(generations=50,  # 公式进化的世代数量  30
                         #function_set=function_set,  # 用于构建和进化公式时使用的函数集
                         parsimony_coefficient=0.001,  # 节俭系数，用于惩罚过于复杂的公式
                         metric='spearman', # 适应度指标，可以用make_fitness自定义
                         p_crossover=0.9,  # 交叉变异概率                     
                         max_samples=0.9,  # 最大采样比例
                         verbose=1,
                         random_state=0,  # 随机数种子
                         n_jobs=-1,  # 并行计算使用的核心数量
                        )

X_train = X_train.fillna(0)
Y_train = Y_train.fillna(0)
st.fit(X_train, Y_train)
df_statistics = pd.DataFrame(st.run_details_)

# 画学习曲线
duration = df_statistics['generation_time'].sum() / 60
x = df_statistics['generation']
plt.plot(x, df_statistics['average_fitness'], label='average')
plt.plot(x, df_statistics['best_fitness'], label='best')
plt.plot(x, df_statistics['best_oob_fitness'], label='best_oob')
plt.title('Learning Curve of Fitness, 耗时:%.0fmin' % duration)
plt.tight_layout()
plt.legend(loc='best')
plt.show()

print(st)


C:\Users\DELL\.conda\envs\pytorch2\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    26.65      0.000601871       15       0.00251141       0.00069142    815.98m
   1     2.99      0.000989435        5       0.00221399       0.00304945    778.37m
   2     1.13      0.000972873        3       0.00199479       0.00061029    812.01m
   3     1.02       0.00138768        1       0.00211135       0.00486355    715.57m
